In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 976.9/976.9 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
# prompt: connect to drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import numpy as np
from ultralytics import YOLO
from tensorflow.keras.models import load_model
from PIL import Image
from tensorflow.keras.preprocessing import image
import tensorflow as tf

def softmax(x):
    """Compute softmax values for each set of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
MODEL_PATHS = {
    'yolo': '/content/drive/MyDrive/Manufacturing defects/best_model (1).pt',
    'cnn': '/content/drive/MyDrive/Manufacturing defects/defect_classifier_cnn.h5',
    'effnet': '/content/drive/MyDrive/Manufacturing defects/EFF_net.h5'
}

In [ ]:
MODEL_WEIGHTS = {
    'yolo': 0.966 / (0.966 + 0.74 + 0.86),
    'cnn': 0.74 / (0.966 + 0.74 + 0.86),
    'effnet': 0.86 / (0.966 + 0.74 + 0.86)
}

In [ ]:
DEFECT_TYPES = ['corner defect', 'dent', 'scratch', 'spidol mark']
SEVERITY_LEVELS = {
    'Minor': (0.0, 0.5),
    'Moderate': (0.5, 0.8),
    'Severe': (0.8, 1.0)
}

In [ ]:
def _ensemble_prediction(self, yolo_conf, cnn_pred, effnet_pred):
        """Robust weighted average ensemble fusion"""
        try:
            # Process and align predictions
            cnn_scores = self._map_predictions(cnn_pred, 'cnn')
            effnet_scores = self._map_predictions(effnet_pred, 'effnet')

            # Ensure we have scores for all DEFECT_TYPES
            if len(cnn_scores) != len(DEFECT_TYPES) or len(effnet_scores) != len(DEFECT_TYPES):
                raise ValueError(f"Prediction shape mismatch. CNN: {len(cnn_scores)}, "
                              f"EfficientNet: {len(effnet_scores)}, Expected: {len(DEFECT_TYPES)}")

            # Weighted average calculation
            weighted_scores = (
                MODEL_WEIGHTS['yolo'] * yolo_conf * np.ones(len(DEFECT_TYPES)) +
                MODEL_WEIGHTS['cnn'] * cnn_scores +
                MODEL_WEIGHTS['effnet'] * effnet_scores
            )

            # Get final prediction
            defect_idx = np.argmax(weighted_scores)
            confidence = np.max(weighted_scores)

            return {
                'type': DEFECT_TYPES[defect_idx],
                'confidence': float(confidence),
                'severity': self._calculate_severity(confidence),
                'model_contributions': {
                    'yolo': float(yolo_conf),
                    'cnn': float(np.max(cnn_scores)),
                    'effnet': float(np.max(effnet_scores))
                },
                'raw_scores': {
                    'cnn': cnn_scores.tolist(),
                    'effnet': effnet_scores.tolist()
                }
            }

        except Exception as e:
            print(f"Ensemble fusion error: {str(e)}")
            # Fallback to YOLO-only prediction
            return {
                'type': DEFECT_TYPES[0],
                'confidence': float(yolo_conf),
                'severity': self._calculate_severity(yolo_conf),
                'model_contributions': {
                    'yolo': float(yolo_conf),
                    'cnn': 0.0,
                    'effnet': 0.0
                },
                'raw_scores': {
                    'cnn': [],
                    'effnet': []
                },
                'error': str(e)
            }

In [ ]:
class DefectDetector:
    def __init__(self):
        """Initialize all models with proper configuration"""
        self.yolo = YOLO(MODEL_PATHS['yolo'])
        self.cnn = load_model(MODEL_PATHS['cnn'], compile=False)
        self.effnet = load_model(MODEL_PATHS['effnet'], compile=False)
        self._warmup_models()

    def _warmup_models(self):
        """Initialize models with dummy data to prevent cold-start issues"""
        dummy_cnn = np.random.rand(1, 224, 224, 3).astype(np.float32)
        dummy_effnet = np.random.rand(1, 256, 256, 3).astype(np.float32)
        _ = self.cnn.predict(dummy_cnn)
        _ = self.effnet.predict(dummy_effnet)

    def _preprocess_image(self, img_array, model_type):
        """Model-specific preprocessing"""
        img = Image.fromarray(img_array)
        target_size = (224, 224) if model_type == 'cnn' else (256, 256)
        img = img.resize(target_size)
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0) / 255.0
        return img.astype(np.float32)

    def _calculate_severity(self, confidence):
        """Determine severity based on confidence score"""
        for severity, (min_val, max_val) in SEVERITY_LEVELS.items():
            if min_val <= confidence < max_val:
                return severity
        return 'Minor'

    def _map_predictions(self, pred, model_name):
        """Map model predictions to our DEFECT_TYPES"""
        return softmax(np.array(pred[0]))

    def _ensemble_prediction(self, yolo_conf, cnn_pred, effnet_pred):
        """Weighted average ensemble fusion"""
        try:
            cnn_scores = self._map_predictions(cnn_pred, 'cnn')
            effnet_scores = self._map_predictions(effnet_pred, 'effnet')

            weighted_scores = (
                MODEL_WEIGHTS['yolo'] * yolo_conf * np.ones(len(DEFECT_TYPES)) +
                MODEL_WEIGHTS['cnn'] * cnn_scores +
                MODEL_WEIGHTS['effnet'] * effnet_scores
            )

            defect_idx = np.argmax(weighted_scores)
            confidence = np.max(weighted_scores)

            return {
                'type': DEFECT_TYPES[defect_idx],
                'confidence': float(confidence),
                'severity': self._calculate_severity(confidence),
                'model_contributions': {
                    'yolo': float(yolo_conf),
                    'cnn': float(np.max(cnn_scores)),
                    'effnet': float(np.max(effnet_scores))
                }
            }
        except Exception as e:
            print(f"Ensemble error: {str(e)}")
            return {
                'type': DEFECT_TYPES[0],
                'confidence': float(yolo_conf),
                'severity': self._calculate_severity(yolo_conf),
                'model_contributions': {
                    'yolo': float(yolo_conf),
                    'cnn': 0.0,
                    'effnet': 0.0
                }
            }

    def detect_defects(self, image_path):
        """Complete defect detection pipeline"""
        try:
            img = np.array(Image.open(image_path).convert('RGB'))
            yolo_results = self.yolo(img)
            defects = []

            for result in yolo_results:
                for box, conf in zip(result.boxes.xyxy.cpu().numpy(),
                                   result.boxes.conf.cpu().numpy()):
                    x1, y1, x2, y2 = map(int, box)
                    roi = img[y1:y2, x1:x2]

                    if roi.size == 0:
                        continue

                    cnn_input = self._preprocess_image(roi, 'cnn')
                    effnet_input = self._preprocess_image(roi, 'effnet')

                    cnn_pred = self.cnn.predict(cnn_input, verbose=0)
                    effnet_pred = self.effnet.predict(effnet_input, verbose=0)

                    ensemble_result = self._ensemble_prediction(conf, cnn_pred, effnet_pred)

                    defects.append({
                        'bbox': [x1, y1, x2, y2],
                        **ensemble_result
                    })

            return defects
        except Exception as e:
            print(f"Detection error: {str(e)}")
            return []

In [ ]:
def detect_defects(self, image_path):
        """Complete defect detection pipeline with error handling"""
        try:
            img = np.array(Image.open(image_path).convert('RGB'))
            yolo_results = self.yolo(img)
            defects = []

            for result in yolo_results:
                for box, conf in zip(result.boxes.xyxy.cpu().numpy(),
                                   result.boxes.conf.cpu().numpy()):
                    x1, y1, x2, y2 = map(int, box)
                    roi = img[y1:y2, x1:x2]

                    if roi.size == 0:
                        continue

                    # Get predictions from both models
                    cnn_input = self._preprocess_image(roi, 'cnn')
                    effnet_input = self._preprocess_image(roi, 'effnet')
                    cnn_pred = self.cnn.predict(cnn_input, verbose=0)
                    effnet_pred = self.effnet.predict(effnet_input, verbose=0)

                    # Ensemble fusion
                    ensemble_result = self._ensemble_prediction(conf, cnn_pred, effnet_pred)

                    defects.append({
                        'bbox': [x1, y1, x2, y2],
                        **ensemble_result
                    })

            return defects

        except Exception as e:
            print(f"Detection pipeline error: {str(e)}")
            return []
if __name__ == "__main__":
    detector = DefectDetector()
    results = detector.detect_defects("/content/drive/MyDrive/Manufacturing defects/datasets new defect/test/images/IMG_20230618_202726_jpg.rf.62a4c10d38d7c24fd4cf3b7ea3b1a304.jpg")

    print("\nENSEMBLE DEFECT DETECTION RESULTS")
    print("="*50)
    for i, defect in enumerate(results, 1):
        print(f"\nDefect {i}:")
        print(f"Type: {defect['type']}")
        print(f"BBox: {defect['bbox']}")
        print(f"Confidence: {defect['confidence']:.4f}")
        print(f"Severity: {defect['severity']}")

        print("\nModel Contributions:")
        print(f"YOLO: {defect['model_contributions']['yolo']:.4f} (weight: {MODEL_WEIGHTS['yolo']:.3f})")
        print(f"CNN: {defect['model_contributions']['cnn']:.2f} (weight: {MODEL_WEIGHTS['cnn']:.3f})")
        print(f"EfficientNet: {defect['model_contributions']['effnet']:.3f} (weight: {MODEL_WEIGHTS['effnet']:.3f})")

        if 'error' in defect:
            print(f"\nWarning: {defect['error']}")
    print("\n" + "="*50)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step

0: 640x640 2 scratchs, 2 spidols, 3284.6ms
Speed: 33.1ms preprocess, 3284.6ms inference, 54.7ms postprocess per image at shape (1, 3, 640, 640)
Ensemble error: operands could not be broadcast together with shapes (4,) (3,) 
Ensemble error: operands could not be broadcast together with shapes (4,) (3,) 
Ensemble error: operands could not be broadcast together with shapes (4,) (3,) 
Ensemble error: operands could not be broadcast together with shapes (4,) (3,) 

ENSEMBLE DEFECT DETECTION RESULTS

Defect 1:
Type: corner defect
BBox: [170, 15, 234, 57]
Confidence: 0.9398
Severity: Severe

Model Contributions:
YOLO: 0.9398 (weight: 0.376)
CNN: 0.00 (weight: 0.288)
EfficientNet: 0.000 (weight: 0.335)

Defect 2:
Type: corner defect
BBox: [294, 186, 344, 213]
Confidence: 0.9304
Severity: Severe

Model Contributions:
YOLO: 0.9304 (weight: 0.376)
CNN: 0.00 (weight: 0.288)
EfficientNet: 0.000 (weight: 0.335)

Defect 3:
Ty

In [ ]:
# prompt: give a code to save and download the model as esemble name

import os

def save_and_download_model(model, model_name="ensemble_model"):
    """Saves the model and provides a download link."""

    # Create a directory to store the model (if it doesn't exist)
    save_dir = "/content/drive/MyDrive/Saved_Models" # Change the path if you wish to
    os.makedirs(save_dir, exist_ok=True)

    # Save the model
    model_path = os.path.join(save_dir, f"{model_name}.h5")  # Or .pt if it's a PyTorch model
    model.save(model_path) # Use model.save_weights() if appropriate
    print(f"Model saved to: {model_path}")

    # Download link (Google Colab specific)
    from google.colab import files
    files.download(model_path)

# Example usage (assuming 'detector' is your ensemble model)
if __name__ == "__main__":
    # ... (your existing code) ...

    save_and_download_model(detector.yolo, model_name="ensemble_model") # assuming detector.yolo is the primary model




Model saved to: /content/drive/MyDrive/Saved_Models/ensemble_model.h5


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [14]:
!pip install ultralytics

In [15]:
# prompt: connect to drive

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
